In [23]:
import pandas as pd
import sqlite3
import xml.etree.ElementTree as ET
from lxml import etree

In [ ]:
import sqlite3
from lxml import etree

def process_xml_to_sqlite(xml_path, db_path, tabel_naam, xml_tag, kolom_definities, kolom_pad,
                          batch_size=100, nested_element_tag=None, nested_offset=0, insert_query=None):

    # Verbind met SQLite en stel optimalisaties in
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("PRAGMA journal_mode = WAL;")
    cursor.execute("PRAGMA synchronous = OFF;")
    
    # Verwijder de tabel als die bestaat en maak een nieuwe aan
    cursor.execute(f"DROP TABLE IF EXISTS {tabel_naam}")
    cursor.execute(f"CREATE TABLE {tabel_naam} ({kolom_definities})")
    conn.commit()

    data = []
    context = etree.iterparse(xml_path, events=("end",), tag=xml_tag)
    
    for event, elem in context:
        if nested_element_tag:
            # Haal ouderdata op (velden 0 tot nested_offset-1)
            ouder_data = tuple((elem.findtext(p) or "").strip() for p in kolom_pad[:nested_offset])
            # Loop over de geneste child-elementen
            for child in elem.findall(nested_element_tag):
                kind_data = tuple((child.findtext(p) or "").strip() for p in kolom_pad[nested_offset:])
                row = ouder_data + kind_data
                data.append(row)
                if len(data) >= batch_size:
                    placeholders = ", ".join("?" for _ in kolom_pad)
                    if insert_query:
                        cursor.executemany(insert_query, data)
                    else:
                        cursor.executemany(f"INSERT OR REPLACE INTO {tabel_naam} VALUES ({placeholders})", data)
                    conn.commit()
                    data.clear()
        else:
            # Geen nested processing: haal alle velden uit hetzelfde element
            row = tuple((elem.findtext(p) or "").strip() for p in kolom_pad)
            data.append(row)
            if len(data) >= batch_size:
                placeholders = ", ".join("?" for _ in kolom_pad)
                if insert_query:
                    cursor.executemany(insert_query, data)
                else:
                    cursor.executemany(f"INSERT OR REPLACE INTO {tabel_naam} VALUES ({placeholders})", data)
                conn.commit()
                data.clear()
        
        # Vrijmaken van geheugen
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    if data:
        placeholders = ", ".join("?" for _ in kolom_pad)
        if insert_query:
            cursor.executemany(insert_query, data)
        else:
            cursor.executemany(f"INSERT OR REPLACE INTO {tabel_naam} VALUES ({placeholders})", data)
        conn.commit()
    
    conn.close()
    print(f"XML succesvol opgeslagen in SQLite: {db_path} (Tabel: {tabel_naam})")


In [ ]:
# Tabeldefinities voor archeologischComplexPreTriply (29 kolommen)
kolom_definities_archeologischComplex = """
    choId TEXT,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    identificatie TEXT,
    samenhangChoId TEXT,
    samenhangTypeCho TEXT,
    samenhangRelatiePreflabel TEXT,
    samenhangRelatieUri TEXT,
    samenhangToelichting TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    zaakId TEXT,
    kennisVertrouwelijkheidPreflabel TEXT,
    kennisVertrouwelijkheidUri TEXT,
    formeelStandpunt TEXT,
    typekenmerkPreflabel TEXT,
    typekenmerkUri TEXT,
    typennaamPreflabel TEXT,
    typennaamUri TEXT,
    typeRegistratiedatum TEXT,
    gebeurtenisBegindateringPreflabel TEXT,
    gebeurtenisBegindateringUri TEXT,
    gebeurtenisEinddateringPreflabel TEXT,
    gebeurtenisEinddateringUri TEXT,
    gebeurtenisNaamPreflabel TEXT,
    gebeurtenisNaamUri TEXT,
    gebeurtenisRegistratiedatum TEXT,
    PRIMARY KEY (choId, kennisregistratienummer)
"""

# De kolom-paden: eerste 11 velden uit het ouder-element, de resterende 18 uit elk <kennisregistratie>
kolom_pad_archeologischComplex = (
    # Oudere velden (11):
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "identificatie",
    "samenhang/isRelatieVan/choId",
    "samenhang/isRelatieVan/typeCho",
    "samenhang/relatieType/preflabel",
    "samenhang/relatieType/uri",
    "samenhang/toelichting",
    # Velden uit <kennisregistratie> (18):
    "kennisregistratienummer",
    "typeKennis",
    "zaakId",
    "vertrouwelijkheid/preflabel",      # uit kennisregistratie
    "vertrouwelijkheid/uri",             # uit kennisregistratie
    "formeelStandpunt",
    "type/typekenmerk/preflabel",
    "type/typekenmerk/uri",
    "type/typenaam/preflabel",
    "type/typenaam/uri",
    "type/registratiedatum",
    "gebeurtenis/datering/begindatering/periode/preflabel",
    "gebeurtenis/datering/begindatering/periode/uri",
    "gebeurtenis/datering/einddatering/periode/preflabel",
    "gebeurtenis/datering/einddatering/periode/uri",
    "gebeurtenis/gebeurtenisnaam/preflabel",
    "gebeurtenis/gebeurtenisnaam/uri",
    "gebeurtenis/registratiedatum"
)

process_xml_to_sqlite(
    xml_path="sourceDatasets/test_archis.xml",
    db_path="database.db",
    tabel_naam="archeologischComplexPreTriply",
    xml_tag="archeologischComplex",
    kolom_definities=kolom_definities_archeologischComplex,
    kolom_pad=kolom_pad_archeologischComplex,
    batch_size=100,
    nested_element_tag="kennisregistratie",  # Voor elk <kennisregistratie>-child
    nested_offset=11                        # De eerste 11 kolom-paden komen uit het ouder-element
)


In [ ]:
kolom_definities_archeologischOnderzoeksgebied = """
    choId TEXT PRIMARY KEY,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    basisregistratieId TEXT,
    bagRelatieId TEXT,
    plaatsnaam TEXT,
    geometrieId TEXT,
    WKT TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    zaakId TEXT,
    kennisVertrouwelijkheidPreflabel TEXT,
    kennisVertrouwelijkheidUri TEXT,
    formeelStandpunt TEXT,
    kennisOmschrijvingPreflabel TEXT,
    kennisOmschrijvingUri TEXT,
    registratiedatum TEXT
"""
kolom_pad_archeologischOnderzoeksgebied = (
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "basisregistratie/basisregistratieId",
    "basisregistratie/bagRelatie/bagRelatieId",
    "basisregistratie/bagRelatie/plaatsnaam",
    "geometrie/geometrieId",
    "geometrie/WKT",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/zaakId",
    "kennisregistratie/vertrouwelijkheid/preflabel",
    "kennisregistratie/vertrouwelijkheid/uri",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/omschrijving/omschrijvingstype/preflabel",
    "kennisregistratie/omschrijving/omschrijvingstype/uri",
    "registratiedatum"
)
process_xml_to_sqlite(
    xml_path="sourceDatasets/test_archis.xml",
    db_path="database.db",
    tabel_naam="archeologischOnderzoeksgebiedPreTriply",
    xml_tag="archeologischOnderzoeksgebied",
    kolom_definities=kolom_definities_archeologischOnderzoeksgebied,
    kolom_pad=kolom_pad_archeologischOnderzoeksgebied,
    batch_size=250
)


In [ ]:
column_defs_vondstlocatie = """
    choId TEXT NOT NULL,
    typeCho TEXT,
    vertrouwelijkheidPreflabel TEXT,
    vertrouwelijkheidUri TEXT,
    objectnummer TEXT,
    basisregistratieId TEXT,
    bagRelatieId TEXT,
    plaatsnaam TEXT,
    locatieAanduidingId TEXT,
    locatietype TEXT,
    locatienaam TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    zaakId TEXT,
    kennisVertrouwelijkheidPreflabel TEXT,
    kennisVertrouwelijkheidUri TEXT,
    formeelStandpunt TEXT,
    omschrijving TEXT,
    omschrijvingPreflabel TEXT,
    omschrijvingUri TEXT,
    registratiedatum TEXT
"""
column_paths_vondstlocatie = (
    "choId",
    "typeCho",
    "vertrouwelijkheid/preflabel",
    "vertrouwelijkheid/uri",
    "objectnummer",
    "basisregistratie/basisregistratieId",
    "basisregistratie/bagRelatie/bagRelatieId",
    "basisregistratie/bagRelatie/plaatsnaam",
    "locatieAanduiding/locatieAanduidingId",
    "locatieAanduiding/locatietype",
    "locatieAanduiding/locatienaam",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/zaakId",
    "kennisregistratie/vertrouwelijkheid/preflabel",
    "kennisregistratie/vertrouwelijkheid/uri",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/omschrijving/tekst",
    "kennisregistratie/omschrijving/omschrijvingstype/preflabel",
    "kennisregistratie/omschrijving/omschrijvingstype/uri",
    "registratiedatum"
)
process_xml_to_sqlite(
    xml_path="sourceDatasets/test_archis.xml",
    db_path="database.db",
    tabel_naam="vondstlocatiePreTriply",
    xml_tag="vondstlocatie", 
    kolom_definities=column_defs_vondstlocatie,
    kolom_pad=column_paths_vondstlocatie,
    batch_size=250
)


In [ ]:
column_defs_grondsporen = """
    choId TEXT NOT NULL,
    typeCho TEXT,
    objectnummer TEXT,
    identificatie TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    formeelStandpunt TEXT,
    omschrijving TEXT,
    omschrijvingPreflabel TEXT,
    omschrijvingUri TEXT,
    registratiedatum TEXT
"""
column_paths_grondsporen = (
    "choId",
    "typeCho",
    "objectnummer",
    "identificatie",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/omschrijving/tekst",
    "kennisregistratie/omschrijving/omschrijvingstype/preflabel",
    "kennisregistratie/omschrijving/omschrijvingstype/uri",
    "registratiedatum"
)
process_xml_to_sqlite(
    xml_path="sourceDatasets/test_archis.xml",
    db_path="database.db",
    tabel_naam="grondsporenPreTriply",
    xml_tag="grondsporen",
    kolom_definities=column_defs_grondsporen,
    kolom_pad=column_paths_grondsporen,
    batch_size=250
)


In [ ]:
column_defs_vondsten = """
    choId TEXT NOT NULL,
    typeCho TEXT,
    objectnummer TEXT,
    identificatie TEXT,
    kennisregistratienummer TEXT,
    typeKennis TEXT,
    formeelStandpunt TEXT,
    omschrijving TEXT,
    omschrijvingPreflabel TEXT,
    omschrijvingUri TEXT,
    registratiedatum TEXT
"""

column_paths_vondsten = (
    "choId",
    "typeCho",
    "objectnummer",
    "identificatie",
    "kennisregistratie/kennisregistratienummer",
    "kennisregistratie/typeKennis",
    "kennisregistratie/formeelStandpunt",
    "kennisregistratie/omschrijving/tekst",
    "kennisregistratie/omschrijving/omschrijvingstype/preflabel",
    "kennisregistratie/omschrijving/omschrijvingstype/uri",
    "registratiedatum"
)

process_xml_to_sqlite(
    xml_path="sourceDatasets/test_archis.xml",
    db_path="database.db",
    tabel_naam="vondstenPreTriply",
    xml_tag="vondsten",  
    kolom_definities=column_defs_vondsten,
    kolom_pad=column_paths_vondsten,
    batch_size=250
)



### Dataverwerking

In [21]:
def load_table_to_dataframe(db_path, table_name):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()
    return df

In [24]:
df_archeologischOnderzoeksgebiedPreTriply = load_table_to_dataframe("database.db", "archeologischOnderzoeksgebiedPreTriply")
df_archeologischComplexPreTriply = load_table_to_dataframe("database.db", "archeologischComplexPreTriply")
df_vondstenPreTriply = load_table_to_dataframe("database.db", "vondstenPreTriply")
df_vondstlocatiePreTriply = load_table_to_dataframe("database.db", "vondstlocatiePreTriply")
df_grondsporenPreTriply = load_table_to_dataframe("database.db", "grondsporenPreTriply")

sqlite3 database.db  
SELECT COUNT(DISTINCT choId) FROM archeologischComplexPreTriply;  
348870


SELECT COUNT(DISTINCT choId) FROM archeologischOnderzoeksgebiedPreTriply;  
102103

In [25]:
df_preTriply = pd.concat([df_archeologischOnderzoeksgebiedPreTriply, df_archeologischComplexPreTriply, df_vondstenPreTriply, df_vondstlocatiePreTriply, df_grondsporenPreTriply])

In [ ]:
df_preTriply = df_preTriply.astype({"zaakId": float})

In [28]:
df_preTriply.to_csv('csv/preTriplyDataframe.csv', index=False)

In [29]:
del df_archeologischOnderzoeksgebiedPreTriply, df_archeologischComplexPreTriply, df_vondstenPreTriply, df_vondstlocatiePreTriply, df_grondsporenPreTriply